In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#collection_name = 'week8_cora_full_ensemble'
collection_name = 'week9_cora_full_dropout'

collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

Number of finished experiments : 13


In [3]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [4]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [5]:
print([k for k in experiment['result'].keys() if 'auroc' in k])

['auroc_total-predictive-entropy_loc', 'auroc_max-score_loc', 'auroc_expected-softmax-entropy_loc', 'auroc_mutual-information_loc', 'auroc_predicted-class-variance_loc', 'auroc_total-predictive-entropy_loc-no-edges', 'auroc_max-score_loc-no-edges', 'auroc_expected-softmax-entropy_loc-no-edges', 'auroc_mutual-information_loc-no-edges', 'auroc_predicted-class-variance_loc-no-edges', 'auroc_total-predictive-entropy_bernoulli', 'auroc_max-score_bernoulli', 'auroc_expected-softmax-entropy_bernoulli', 'auroc_mutual-information_bernoulli', 'auroc_predicted-class-variance_bernoulli', 'auroc_total-predictive-entropy_bernoulli-no-edges', 'auroc_max-score_bernoulli-no-edges', 'auroc_expected-softmax-entropy_bernoulli-no-edges', 'auroc_mutual-information_bernoulli-no-edges', 'auroc_predicted-class-variance_bernoulli-no-edges', 'auroc_total-predictive-entropy_normal', 'auroc_max-score_normal', 'auroc_expected-softmax-entropy_normal', 'auroc_mutual-information_normal', 'auroc_predicted-class-varianc

In [6]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    if any('Operating_Systems' in label for label in experiment['config']['data']['val_labels']):
        ood = 'os'
    else:
        ood = 'ai'
    
    prefix = (ood, remove_other, residual, spectral_norm)
    
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        med = np.median(np.array(v))
        if 'auroc' in k:
            if 'no-edges' in k:
                no_edges = True
                k = k.replace('_no-edges', '')
                k = k.replace('-no-edges', '')
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            k, ood_type = '_'.join(k_tokens[:-1]), k_tokens[-1]
            prefix_edges = prefix + (no_edges, ood_type,)
                
            if not 'no' in k and ':' in k:
                continue
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'mean')] = [mean]
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'std')] = [std]
            #d[prefix_edges + ('feature density', k.replace('auroc:', ''), 'median')] = [med]
        elif k.startswith('accuracy'):
            k_tokens = k.split('_')
            if k_tokens[1] in ('id', 'ood'):
                k_tokens = ['_'.join(k_tokens[:2])] + k_tokens[2:]
            if 'no-edges' in k_tokens[-1]:
                k_tokens[-1] = k_tokens[-1].replace('no-edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            k_tokens = [token for token in k_tokens if len(token) > 0]
            if len(k_tokens) == 3:
                ood_type = k_tokens[-1]
                k_tokens = k_tokens[:-1]
            else:
                #ood_type = ''
                continue
            name, dataset = k_tokens
            
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'mean')] = [mean]
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'std')] = [std]
            #d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'med')] = [med]
        elif k.startswith('ece') and False:
            if 'no_edges' in k:
                k = k.replace('no_edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            d[prefix + (no_edges, '', 'ece', '', 'mean')] = [mean]
            d[prefix + (no_edges, '', 'ece', '', 'std')] = [std]
            
            #print(name, dataset, prefix + (no_edges, ood_type,))
            
            
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'median')] = [med]
                
df = pd.DataFrame(d).T.sort_index()

In [7]:
df

0
ai False False True False bernoulli AUC-ROC  expected softmax entropy mean  0.625079
                                                                      std   0.071041
                                             logit energy             mean  0.604104
                                                                      std   0.063007
                                             max score                mean  0.582494
...                                                                              ...
os True  True  True True  normal    accuracy                          std   0.035966
                                             id                       mean  0.670159
                                                                      std   0.039846
                                             ood                      mean  0.117143
                                                                      std   0.092317

[1404 rows x 1 columns]

In [8]:
df = df.reset_index((0, 1, 2, 3, 5, -1))

In [9]:
df.shape

(1404, 7)

In [10]:
df

level_0  level_1  level_2  level_3  \
False AUC-ROC  expected softmax entropy      ai    False    False     True   
               expected softmax entropy      ai    False    False     True   
               logit energy                  ai    False    False     True   
               logit energy                  ai    False    False     True   
               max score                     ai    False    False     True   
...                                         ...      ...      ...      ...   
True  accuracy                               os     True     True     True   
               id                            os     True     True     True   
               id                            os     True     True     True   
               ood                           os     True     True     True   
               ood                           os     True     True     True   

                                           level_5 level_8         0  
False AUC-ROC  expected softmax entropy  bernoulli    mean  0.625079  
               expected softmax entropy  bernoulli     std  0.071041  
               logit energy              bernoulli    mean  0.604104  
               logit energy              bernoulli     std  0.063007  
               max score                 bernoulli    mean  0.582494  
...                                            ...     ...       ...  
True  accuracy                              normal     std  0.035966  
               id                           normal    mean  0.670159  
               id                           normal     std  0.039846  
               ood                          normal    mean  0.117143  
               ood                          normal     std  0.092317  

[1404 rows x 7 columns]

In [11]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_3', 'level_5', 'level_8']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [12]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('OOD Classes', 'Remove OOD', 'Residual', 'Spectral Norm', 'OOD Experiment', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('Remove-Edges', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T

In [13]:
exp_sizes = [
    1, # ood data
    1, # remove ood vertices from train
    2, # residual,
    2, # Spectral norm,
    3, # ood type
    2, # stats
]
exp_size = reduce(lambda x, y: x * y, exp_sizes)
exp_size

24

In [14]:


df_cat.iloc[:, 0 * exp_size : 1 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                         False                   
Residual                                           False                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.63  0.73  2.15e-01   
                      logit energy                  0.60  0.67  2.20e-01   
                      max score                     0.58  0.79  3.91e-01   
                      mutual information            0.39  0.45  7.94e-01   
                      predicted class variance      0.40  0.31  7.48e-01   
                      total predictive entropy      0.59  0.77  3.11e-01   
             accuracy                               0.76  0.78  7.37e-01   
                      id                            0.77  0.76  7.69e-01   
                      ood                           0.70   NaN  4.46e-01   
True         AUC-ROC  expected softmax entropy      0.98  0.69  4.92e-03   
                      logit energy                  0.92  0.65  1.20e-03   
                      max score                     0.90  0.71  1.08e-01   
                      mutual information            0.03  0.34  9.99e-01   
                      predicted class variance      0.06  0.32  9.91e-01   
                      total predictive entropy      0.89  0.71  5.20e-02   
             accuracy                               0.64  0.69  6.39e-01   
                      id                            0.70  0.67  6.97e-01   
                      ood                           0.15   NaN  1.14e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy  6.37e-01  0.82  2.13e-01   
                      logit energy              7.00e-01  0.80  3.90e-02   
                      max score                 6.01e-01  0.82  5.52e-01   
                      mutual information        4.51e-01  0.64  8.17e-01   
                      predicted class variance  4.69e-01  0.50  8.25e-01   
                      total predictive entropy  6.13e-01  0.81  4.52e-01   
             accuracy                           8.03e-01  0.81  7.36e-01   
                      id                        8.11e-01  0.77  7.95e-01   
                      ood                       7.34e-01   NaN  2.06e-01   
True         AUC-ROC  expected softmax entropy  9.97e-01  0.63  8.96e-03   
                      logit energy              9.98e-01  0.69  6.80e-04   
                      max score                 9.88e-01  0.71  2.81e-01   
                      mutual information        7.14e-03  0.53  8.57e-01   
                      predicted class variance  9.14e-03  0.38  8.32e-01   
                      total predictive entropy  9.93e-01  0.69  9.38e-02   
             accuracy                           6.65e-01  0.72  6.62e-01   
                      id                        7.23e-01  0.69  7.21e-01   
                      ood                       1.40e-01   NaN  1.31e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                      

In [15]:

df_cat.iloc[:, 1 * exp_size : 2 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                         True                    
Residual                                           False                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.63  0.77  2.14e-01   
                      logit energy                  0.63  0.71  1.99e-01   
                      max score                     0.62  0.84  4.19e-01   
                      mutual information            0.38  0.43  8.33e-01   
                      predicted class variance      0.39  0.29  7.66e-01   
                      total predictive entropy      0.61  0.82  3.48e-01   
             accuracy                               0.81  0.81  7.69e-01   
                      id                            0.81  0.81  8.09e-01   
                      ood                           0.77   NaN  4.14e-01   
True         AUC-ROC  expected softmax entropy      0.97  0.71  8.46e-03   
                      logit energy                  0.92  0.67  1.41e-03   
                      max score                     0.89  0.76  1.37e-01   
                      mutual information            0.03  0.33  9.98e-01   
                      predicted class variance      0.06  0.29  9.86e-01   
                      total predictive entropy      0.90  0.75  7.97e-02   
             accuracy                               0.65  0.70  6.56e-01   
                      id                            0.71  0.72  7.13e-01   
                      ood                           0.13   NaN  1.46e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy  6.62e-01  0.84  2.52e-01   
                      logit energy              7.10e-01  0.84  4.38e-02   
                      max score                 6.19e-01  0.86  6.38e-01   
                      mutual information        4.78e-01  0.71  8.81e-01   
                      predicted class variance  4.97e-01  0.53  8.89e-01   
                      total predictive entropy  6.33e-01  0.86  5.21e-01   
             accuracy                           8.22e-01  0.82  7.55e-01   
                      id                        8.25e-01  0.84  8.16e-01   
                      ood                       7.94e-01   NaN  2.06e-01   
True         AUC-ROC  expected softmax entropy  9.99e-01  0.67  1.34e-02   
                      logit energy              9.95e-01  0.71  5.90e-04   
                      max score                 9.91e-01  0.75  3.18e-01   
                      mutual information        5.65e-03  0.54  8.85e-01   
                      predicted class variance  7.44e-03  0.35  8.59e-01   
                      total predictive entropy  9.96e-01  0.74  1.20e-01   
             accuracy                           6.78e-01  0.73  6.74e-01   
                      id                        7.36e-01  0.74  7.33e-01   
                      ood                       1.60e-01   NaN  1.49e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                      

In [16]:

df_cat.iloc[:, 2 * exp_size : 3 * exp_size : 2]

OOD Classes                                           os                  \
Remove OOD                                         False                   
Residual                                           True                    
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.74  0.83  1.83e-02   
                      logit energy                  0.63  0.72  5.15e-02   
                      max score                     0.68  0.86  1.30e-01   
                      mutual information            0.27  0.34  9.91e-01   
                      predicted class variance      0.32  0.28  9.56e-01   
                      total predictive entropy      0.69  0.86  5.51e-02   
             accuracy                               0.74  0.76  6.94e-01   
                      id                            0.76  0.73  7.47e-01   
                      ood                           0.62   NaN  2.14e-01   
True         AUC-ROC  expected softmax entropy      0.95  0.70  9.75e-04   
                      logit energy                  0.80  0.67  8.16e-04   
                      max score                     0.84  0.75  2.64e-02   
                      mutual information            0.08  0.37  9.99e-01   
                      predicted class variance      0.16  0.33  9.89e-01   
                      total predictive entropy      0.80  0.74  6.51e-03   
             accuracy                               0.59  0.63  5.93e-01   
                      id                            0.64  0.61  6.41e-01   
                      ood                           0.15   NaN  1.60e-01   

OOD Classes                                                               \
Remove OOD                                         True                    
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy  6.00e-01  0.91  4.59e-01   
                      logit energy              6.30e-01  0.93  1.72e-01   
                      max score                 5.90e-01  0.90  6.66e-01   
                      mutual information        5.25e-01  0.53  8.23e-01   
                      predicted class variance  5.33e-01  0.39  7.96e-01   
                      total predictive entropy  5.94e-01  0.89  6.15e-01   
             accuracy                           8.23e-01  0.83  7.97e-01   
                      id                        8.25e-01  0.85  8.28e-01   
                      ood                       8.06e-01   NaN  5.17e-01   
True         AUC-ROC  expected softmax entropy  9.96e-01  0.70  2.98e-01   
                      logit energy              9.99e-01  0.77  2.02e-03   
                      max score                 9.83e-01  0.76  5.13e-01   
                      mutual information        3.61e-03  0.44  8.54e-01   
                      predicted class variance  4.76e-03  0.24  7.93e-01   
                      total predictive entropy  9.87e-01  0.74  4.71e-01   
             accuracy                           6.94e-01  0.75  6.93e-01   
                      id                        7.54e-01  0.75  7.55e-01   
                      ood                       1.57e-01   NaN  1.40e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                      

In [17]:

df_cat.iloc[:, 3 * exp_size : 4 * exp_size : 2]

OOD Classes                                           os                
Remove OOD                                          True                
Residual                                            True                
Spectral Norm                                       True                
OOD Experiment                                 bernoulli   loc    normal
Stat                                                mean  mean      mean
Remove-Edges                                                            
False        AUC-ROC  expected softmax entropy      0.75  0.81  2.99e-02
                      logit energy                  0.69  0.75  6.59e-02
                      max score                     0.69  0.87  1.68e-01
                      mutual information            0.28  0.36  9.88e-01
                      predicted class variance      0.32  0.28  9.53e-01
                      total predictive entropy      0.71  0.86  7.37e-02
             accuracy                               0.79  0.79  7.25e-01
                      id                            0.80  0.78  7.85e-01
                      ood                           0.67   NaN  1.89e-01
True         AUC-ROC  expected softmax entropy      0.93  0.72  1.13e-03
                      logit energy                  0.79  0.70  3.20e-03
                      max score                     0.82  0.76  4.07e-02
                      mutual information            0.09  0.34  9.99e-01
                      predicted class variance      0.14  0.30  9.83e-01
                      total predictive entropy      0.83  0.76  9.16e-03
             accuracy                               0.62  0.67  6.15e-01
                      id                            0.67  0.68  6.70e-01
                      ood                           0.14   NaN  1.17e-01